RF1 — Alta y gestión de libros 

Registrar libros con atributos mínimos: isbn, titulo, autor, ejemplares_totales. 
Consultar disponibilidad (ejemplares_disponibles). 
Buscar por título o autor. 

In [ ]:
# Defino una clase llamada libro
class libro():
    
    # Este es el constructor, se ejecuta cuando creo un objeto libro
    def __init__(self, isbn, titulo, autor, ejemplares):
        self.isbn = isbn              # Guardamos el ISBN del libro
        self.titulo = titulo          # Guardamos el título del libro
        self.autor = autor            # Guardamos el autor del libro
        self.ejemplares = ejemplares  # Cantidad de libros disponibles
        self.librosPrestado = 0       # Al inicio no hay libros prestados

    # Método para ver cuántos ejemplares hay disponibles
    def ejemplares_disponibles(self):
        return self.ejemplares

    # Método que revisa si hay al menos un libro disponible
    def disponible(self):
        return self.ejemplares > 0   # Regresa True o False

    # Método para prestar un libro 
    def reducir_ejemplares(self):
        if self.ejemplares > 0:          # Verificamos que haya libros
            self.ejemplares -= 1         # Restamos un ejemplar
            self.librosPrestado += 1     # Sumamos uno a los prestados
            return True                 # Se pudo prestar
        return False                    # No había libros disponibles

    # Método para devolver un libro
    def devolver_libro(self):
        if self.librosPrestado > 0:      # Verificamos que haya libros prestados
            self.ejemplares += 1         # Regresamos un ejemplar
            self.librosPrestado -= 1     # Restamos uno de prestados
            return True                 # Se devolvió correctamente
        return False                    # No había libros para devolver

In [19]:
libro1=libro(12932,'chuyito','javier',2)

In [20]:
libro1.reducir_ejemplares()


True

In [21]:
libro1.ejemplares_disponibles()

1

In [22]:
libro1.devolver_libro()

True

RF2 — Registro de usuarios 

Alta de usuario con id_usuario (único) y nombre. 
Consultar usuarios existentes. 

In [23]:
# Defino una clase llamada usuario
class usuario():
    
    # Constructor: se ejecuta cuando creo un nuevo usuario
    def __init__(self, id_usuario, nombre):
        self.id_usuario = id_usuario          # ID único del usuario
        self.nombre = nombre                  # Nombre del usuario
        self.prestamos_activos = []           # Lista de libros prestados (ISBN)

    # Método para verificar si puede pedir más libros
    def puede_pedir_prestado(self):
        # Solo puede tener máximo 3 libros prestados
        return len(self.prestamos_activos) < 3

    # Método para registrar un préstamo
    def registrar_prestamo(self, isbn):
        # Primero verifico si puede pedir prestado
        if self.puede_pedir_prestado():
            self.prestamos_activos.append(isbn)  # Agrego el ISBN a la lista
            return True                          # Préstamo exitoso
        return False                             # No puede pedir más libros

    # Método para registrar la devolución de un libro
    def registrar_devolucion(self, isbn):
        # Verifico si ese libro está en la lista de préstamos
        if isbn in self.prestamos_activos:
            self.prestamos_activos.remove(isbn)  # Elimino el ISBN de la lista
            return True                          # Devolución exitosa
        return False                             # Ese libro no estaba prestado

In [24]:
# Clase que se encarga de gestionar toda la biblioteca
class GestionBiblioteca:
    
    # Constructor: recibe listas de libros y usuarios
    def __init__(self, libros, usuarios):
        self.libros = libros                      # Lista de objetos libro
        self.usuarios = usuarios                  # Lista de objetos usuario
        self.prestamos_registro = []              # Lista para guardar historial de préstamos
        self.sig_id = len(usuarios) + 1           # ID siguiente para nuevos usuarios

    # Método para buscar un libro por su ISBN
    def buscar_libro(self, isbn):
        for l in self.libros:                     # Recorro la lista de libros
            if l.isbn == isbn:                   # Si encuentro coincidencia
                return l                         # Regreso el objeto libro
        return None                              # Si no lo encuentra

    # Método para buscar un usuario por su ID
    def buscar_usuario(self, id_u):
        for u in self.usuarios:                  # Recorro la lista de usuarios
            if u.id_usuario == id_u:             # Si coincide el ID
                return u                         # Regreso el usuario
        return None                              # Si no existe

    # Método para realizar un préstamo
    def realizar_prestamo(self, isbn, id_u, fecha):
        libro = self.buscar_libro(isbn)          # Busco el libro
        usuario = self.buscar_usuario(id_u)      # Busco el usuario

        if libro is None:
            return "No existe el libro"
        if usuario is None:
            return "No existe el usuario"

        # Verifico si el usuario ya tiene ese libro
        if isbn in usuario.prestamos_activos:
            return "Ya tiene ese libro"

        # Verifico si puede pedir más libros
        if not usuario.puede_pedir_prestado():
            return "Ya tiene muchos libros"

        # Intento reducir ejemplares (prestar libro)
        if libro.reducir_ejemplares():
            usuario.registrar_prestamo(isbn)                     # Registro préstamo en usuario
            self.prestamos_registro.append((isbn, id_u, fecha))  # Guardo en historial
            return "Prestado"
        else:
            return "No hay libros"                               # No había disponibles

    # Método para devolver un libro
    def realizar_devolucion(self, isbn, id_u):
        libro = self.buscar_libro(isbn)          # Busco el libro
        usuario = self.buscar_usuario(id_u)      # Busco el usuario

        if libro is None or usuario is None:
            return "Error"                       # Si algo no existe

        # Intento registrar devolución en el usuario
        if usuario.registrar_devolucion(isbn):
            libro.devolver_libro()               # Regreso el ejemplar al libro
            return "Devuelto"
        else:
            return "No lo tenia"                 # El usuario no tenía ese libro

    # Método para obtener los 3 libros más prestados
    def top_3_prestados(self):
        lista = self.libros.copy()               # Copio la lista original

        # Ordenamiento
        for i in range(len(lista)):
            for j in range(len(lista)):
                # Comparo cuántas veces se ha prestado cada libro
                if lista[i].librosPrestado > lista[j].librosPrestado:
                    temp = lista[i]              # Intercambio posiciones
                    lista[i] = lista[j]
                    lista[j] = temp

        return lista[0:3]                        # Regreso los 3 primeros

In [25]:
biblio = [
    libro(12932, 'Cien Años de Soledad', 'García Márquez', 2),
    libro(99541, 'El Quijote', 'Cervantes', 1),
    libro(77777, 'Rayuela', 'Cortázar', 3)
]

usuarios_init = [
    usuario(1, 'Ana'),
    usuario(2, 'Luis')
]

gestionador = GestionBiblioteca(biblio, usuarios_init)



In [ ]:
# Ciclo infinito para que el programa siga corriendo hasta que el usuario salga
while True:
    
    # Menú de opciones
    print("\n1 Alta usuario")
    print("2 Listar usuarios")
    print("3 Prestar libro")
    print("4 Devolver libro")
    print("5 Top 3 libros más prestados")
    print("6 Salir")

    op = input("Opcion: ")   # Pido al usuario que elija una opción

    # Opción 1: dar de alta un usuario
    if op == "1":
        nom = input("Nombre: ")   # Pido el nombre
        nuevo = usuario(gestionador.sig_id, nom)   # Creo un nuevo usuario
        gestionador.usuarios.append(nuevo)         # Lo agrego a la lista
        print("ID asignado:", gestionador.sig_id)  # Muestro el ID generado
        gestionador.sig_id += 1                    # Incremento el ID para el siguiente

    # Opción 2: mostrar todos los usuarios
    elif op == "2":
        for u in gestionador.usuarios:   # Recorro la lista de usuarios
            # Imprimo ID, nombre y cantidad de préstamos activos
            print(u.id_usuario, u.nombre, "Prestamos:", len(u.prestamos_activos))

    # Opción 3: prestar un libro
    elif op == "3":
        isbn = int(input("ISBN: "))          # Pido el ISBN del libro
        uid = int(input("ID usuario: "))     # Pido el ID del usuario
        fecha = input("Fecha: ")             # Pido la fecha
        # Llamo al método de préstamo y muestro el resultado
        print(gestionador.realizar_prestamo(isbn, uid, fecha))

    # Opción 4: devolver un libro
    elif op == "4":
        isbn = int(input("ISBN: "))          # Pido el ISBN
        uid = int(input("ID usuario: "))     # Pido el ID del usuario
        # Llamo al método de devolución y muestro el resultado
        print(gestionador.realizar_devolucion(isbn, uid))

    # Opción 5: mostrar los 3 libros más prestados
    elif op == "5":
        top = gestionador.top_3_prestados()  # Obtengo la lista
        i = 1
        for l in top:   # Recorro los libros del top
            # Imprimo posición, título y número de préstamos
            print(i, l.titulo, "-", l.librosPrestado, "prestamos")
            i += 1

    # Opción 6: salir del programa
    elif op == "6":
        break   # Termina el ciclo y el programa

    # Si el usuario escribe algo inválido
    else:
        print("No valido")


1 Alta usuario
2 Listar usuarios
3 Prestar libro
4 Devolver libro
5 Top 3 libros más prestados
6 Salir
1 Ana Prestamos: 0
2 Luis Prestamos: 0

1 Alta usuario
2 Listar usuarios
3 Prestar libro
4 Devolver libro
5 Top 3 libros más prestados
6 Salir
